### Categorizing the original dataset into five food categories. 

In [2]:
import pandas as pd
import re

In [16]:
data = pd.read_csv(r'Data/NHANES Individual Food Consumption Day 1 (Reduced).csv')
FoodCodes = pd.read_csv(r'Data/USDA Food Codes-edited.csv')
demog = pd.read_csv(r'Data/NHANES Demographics.csv')

data.rename(columns={"DR1IFDCD": "Food code"},inplace=True)
FoodCodes.rename(columns={"Main food description": "Description"},inplace=True)
data = data.drop(data.columns[3:], axis=1)
demog = demog.drop(demog.columns[5:], axis=1)

In [10]:
demog = demog[(demog['RIAGENDR'] == 2) & (demog['RIDAGEYR'] >=19)& (demog['RIDAGEYR'] <= 59)]

In [11]:
FoodItems = ["milk","cream","ice cream","cheese","sausage","fish","stew","frozen meal","beans","nuts","seeds","bread","biscuit","cake","crackers","pasta","cereal","pizza",
             "citrus fruit","other fruit","juice","potato","greens","tomato","vegetable","salad dressing", "caffeinated drinks", "smoothie", "desserts","soup","meat","chicken",
             "egg meals","fast food","squash","oil" ]

# pattern = '|'.join(FoodItems)
pattern = '^(' + '|'.join(FoodItems) + ')'

# Extract matching strings from text column and save in new column
FoodCodes["MainFood"] = FoodCodes.Description.str.extract(pattern, flags=re.IGNORECASE)

# Lower case all items
FoodCodes['MainFood'] = FoodCodes['MainFood'].str.lower()

# Drop the NaN rows
FoodCodes=FoodCodes.dropna(subset=['MainFood'])

In [12]:
mergee = data.merge(FoodCodes, left_on='Food code',right_on='Food code').reindex(columns=['SEQN', 'Food code', 'DR1IGRMS', 'MainFood'])
mergeee = mergee.merge(demog, left_on='SEQN',right_on='SEQN').reindex(columns=['SEQN', 'Food code', 'DR1IGRMS', 'MainFood'])

In [13]:
counts = mergeee.groupby(["SEQN", "MainFood"]).count()
counts.rename(columns={'DR1IGRMS': 'count'},inplace=True)
counts.drop(columns=data.columns, errors='ignore',inplace=True)
counts = counts.unstack(level=1)
# counts = counts.drop(counts.index[20:], axis=0)
counts.fillna(0,inplace=True)

# # Drop the columns that are consumed by less that 3 people
# counts_reduced = counts.dropna(thresh=3,axis=1)
# counts_reduced.fillna(0,inplace=True)
# counts_reduced

In [14]:
final = counts["count"].copy()
final['Fruits'] = final['citrus fruit'] + final["other fruit"]  + final["smoothie"] + final["juice"]  
final['Vegetables'] =  final["squash"] + final["vegetable"]  + final["greens"] + final["potato"]  + final["tomato"]
final['Grains'] = final["cake"] + final["crackers"]  + final["pasta"] + final["cereal"]  + final["bread"] + final["biscuit"]
final['Protein'] = final["fish"] + final["egg meals"]  + final["sausage"] + final["meat"]  + final["chicken"] + final["stew"] + final["beans"]  + final["nuts"] + final["seeds"]
final['Dairy'] = final["milk"] + final["cheese"]
final = final.drop(final.columns[0:35], axis=1)
final

MainFood,Fruits,Vegetables,Grains,Protein,Dairy
SEQN,,,,,
83735,0.0,0.0,1.0,1.0,0.0
83736,2.0,1.0,0.0,0.0,0.0
83742,4.0,1.0,1.0,2.0,1.0
83752,1.0,2.0,3.0,2.0,1.0
83757,1.0,4.0,2.0,3.0,1.0
...,...,...,...,...,...
93665,1.0,2.0,5.0,1.0,0.0
93676,1.0,3.0,2.0,1.0,1.0
93677,5.0,2.0,1.0,3.0,2.0


### For deriving the known constraints, the above dataset is considered as input.

In [198]:
# final.to_csv(r'test_F1959.csv', index=True)